# Backtesting Reversal Strategies and Momentum Strategies on a Portfolio of S&P 500 Constituents

The aim of this python programme is to backtest reversal strategies and momentum strategies performed over a portfolio consisting of S&P 500 index (S&P 500) constituents. In this programme, reversal strategy means buying losers over the past period while momentum strategies means buying winners over the past period. Unlike those mentioned in academic journals, these strategies do not involve short-selling, considering the fact that most individuals are not able to perform short-selling. In this programme, we backtest a series of reversal strategies and a series of momentum strategies (differing by how we choose stocks to form a portfolio), as listed in the following:

1. Reversal strategy of buying losers (top 10 worst-performing stocks) over the past 5 years
2. Reversal strategy of buying losers over the past 3 years
3. Reversal strategy of buying losers over the past 1 year
4. Reversal strategy of buying the worst-performing stocks over the period of "60 months ago to 12 months ago"
5. Reversal strategy of buying losers over the past 1 month
6. Momentum strategy of buying winners (top 10 best-performing stocks) over the past 1 month
7. Momentum strategy of buying winners over the past 3 month
8. Momentum strategy of buying winners over the past 6 month
9. Momentum strategy of buying winners over the past 9 month,
and also the following 2 strategies:
1. Momentum strategy of buying winners over the past 12 month
2. Momentum strategy of buying the best-performing stocks over the period of "12 months ago to 1 month ago"

Every year at year-end (31/12), we rank the return of the constituents over a selected period in the past, based on the strategy we have chosen (e.g. the past 1 year, or the past 6 months, etc., referring to the strategies above). We then form a portfolio consisting of the top 10 stocks in the list, and hold it for 1 year. We compute this actual holding return. 

This programmes automatically downloads the historical price of the constituents from "Yahoo! Finance" website, for calculation and analysis.    

In [817]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

The following code cell is to get a list of "year-end S&P 500 constituents", from 2021 to 2007. 

In [893]:
fhand = open('.\Documents\Stock\Backtests\SP_List.csv', 'r')
sp = []

for line in fhand:
    line = line.rstrip()
    line = line.split(',')
    #if line[0] == '2021':
    splst = []
    for i in line:
        if i != '':
            splst.append(i)
    sp.append(splst)
print(sp)        
        
fhand.close()

[['2022', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'FRC', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'SBNY', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI',

The following code cell can be ignored. It is used to download monthly historical stock price at year-end ONE YEAR AT A TIME, and hence to backtest the performance of the portfolio ONE YEAR AT A TIME. This will be very slow. Use this cell code unless you want to backtest one year at a time. 

In the following example, historical adjusted stock price from 2016-12-01 to 2022-12-31 is downloaded. Then a portfolio consisting of the top 10 stock (based on the return from 2017-01-01 to 2021-12-31, i.e. past 5 years) is formed. In the middle of the code cell, there is a block of code to compute the Beta of each constituent based on adjusted stock price over the past 5 years. Lastly, the holding period (2022-01-01 to 2022-12-31) return is computed.

In [892]:
#Get the monthly adjusted closing stock price of S&P 500 constituents
st = yf.download(splst, 
                      start='2016-12-01', 
                      end='2022-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)

lst = np.log(st)
ri = lst.diff()
ri.index = pd.to_datetime(ri.index)
rihist = ri['Adj Close'].loc['2017-01-01':'2021-12-31'].dropna(axis='columns')
mean = rihist.mean(axis=0)*12
print(mean.sort_values(ascending=True)[0:10])

#Get the monthly adjusted closing stock price of S&P 500
sm = yf.download('^GSPC', 
                      start='2016-12-01', 
                      end='2022-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
lsm = np.log(sm)
rm = lsm.diff()
rm.index = pd.to_datetime(rm.index)
rmhist = rm['Adj Close'].loc['2017-01-01':'2021-12-31']
rmhistnp = rmhist.to_numpy()
rmhistnp = rmhistnp.reshape(-1, 1)

# Create linear regression object
regr = linear_model.LinearRegression()

betas = []
for i in list(rihist.columns):
    rinp = np.array(rihist[i]).reshape(-1, 1)
    rinp = rinp.reshape(60,)
    rinp = rinp.reshape(-1, 1) 
    regr.fit(rmhistnp, rinp)
    beta = regr.coef_
    betas.append((beta, i))
betas.sort(reverse=True)
betalst = []
for i in betas[0:10]:
    betalst.append(i[1])
print(betalst)
    
stocklst = list(mean.sort_values(ascending=True)[0:10].index)
print(stocklst)
st.index = pd.to_datetime(st.index)
print(np.exp(ri['Adj Close'].loc['2022-01-01':'2022-12-31'][betalst].sum(axis=0)))
x = np.log((np.exp(ri['Adj Close'].loc['2022-01-01':'2022-12-31'][betalst].sum(axis=0)).sum(axis=0))/10)
print(x)

#with pd.ExcelWriter('Backtests_by_Python.xlsx') as writer:  
    #st.to_excel(writer, sheet_name='s')
    #ri.to_excel(writer, sheet_name='r')
    #mean.to_excel(writer, sheet_name='miu')


12 Failed downloads:
- NLSN: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- FISV: Data doesn't exist for startDate = 1480568400, endDate = 1672462800
- FBHS: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- FRC: Data doesn't exist for startDate = 1480568400, endDate = 1672462800
- INFO: No timezone found, symbol may be delisted
- SIVB: No timezone found, symbol may be delisted
VTRS   -0.205907
GE     -0.185791
AAL    -0.184227
SLB    -0.171248
CCL    -0.168484
HAL    -0.151862
APA    -0.151500
PARA   -0.149150
NCLH   -0.143629
OXY    -0.139738
dtype: float64
['APA', 'CZR', 'NCLH', 'MRO', 'DVN', 'HAL', 'RCL', 'OXY', 'PENN', 'FANG']
['VTRS', 'GE', 'AAL', 'SLB', 'CCL', 'HAL', 'APA', 'PARA', 'NCLH', 'OX

The following code cell is to do the following things:
1. rank the constituents based on past return (as described before), for every year from 2007 year-end to 2021-year end,
2. carry out portfolio formation (as described before) at year-end,
3. compute the holding period return, for every year from 2008 year-end to 2022-year end.  

For the 11 strategies mentioned at the top, we make changes accordingly in the code cell below, and run this code cell repeatedly in order to complete our backtesting for these 11 strategies.

In the following example, the "Reversal strategy of buying losers over the past 3 years" is backtested. 

In [894]:
fhand = open('.\Documents\Stock\Backtests\SP_List.csv', 'r')
yrlst = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007']
yr = '2014'
result = []
for splst in sp[1:]:
    #for line in fhand:
        #line = line.rstrip()
        #line = line.split(',')
        #if line[0] == y:
            #splst = []
            #for i in line:
                #if i != '':
                    #splst.append(i)
    print(splst)
    
    #Please change the "years" and "months" in the follwing line accordingly, for each strategy.
    st = yf.download(splst, start=str(int(splst[0])-3)+'-12-01', end=str(int(splst[0])+1)+'-12-31', progress=False, interval="1mo",).drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
    lst = np.log(st)
    ri = lst.diff()
    ri.index = pd.to_datetime(ri.index)
    #Please change the "years" and "months" in the follwing line accordingly, for each strategy.
    rihist = ri['Adj Close'].loc[str(int(splst[0])-2)+'-01-01':str(int(splst[0])-0)+'-12-31'].dropna(axis='columns')
    print(rihist.shape)
    mean = rihist.mean(axis=0)*12
    print(mean.sort_values(ascending=True)[0:10])
    stocklst = list(mean.sort_values(ascending=True)[0:10].index)
    print(stocklst)
    st.index = pd.to_datetime(st.index)
    x = np.log((np.exp(ri['Adj Close'].loc[str(int(splst[0])+1)+'-01-01':str(int(splst[0])+1)+'-12-31'][stocklst].sum(axis=0)).sum(axis=0))/10)
    print(x)
    result.append(x)
    print(result)

fhand.close()

['2021', 'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'TWTR', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'FRC', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'SBNY', 'CHRW', 'CDNS', 'CZR', 'PBCT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'GPS', 'COO', 'CPRT', 'GLW', 'CTVA', 'PVH', 'COST', 'CTRA', 'CCI', 


21 Failed downloads:
- HFC: No timezone found, symbol may be delisted
- KSU: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- MXIM: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- FISV: Data doesn't exist for startDate = 1512104400, endDate = 1640926800
- FBHS: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- VAR: No timezone found, symbol may be delisted
- ALXN: No timezone found, symbol may be delisted
- CXO: No timezone found, symbol may be delisted
- 2020: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- TIF: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- FRC: Data doesn't exist for startDate = 1512104400, endDate = 1640926800
- INFO: No timezone found, symbol


38 Failed downloads:
- ETFC: No timezone found, symbol may be delisted
- HFC: No timezone found, symbol may be delisted
- KSU: No timezone found, symbol may be delisted
- VIAB: No timezone found, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- MXIM: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- WCG: No timezone found, symbol may be delisted
- FISV: Data doesn't exist for startDate = 1448946000, endDate = 1577768400
- FBHS: No timezone found, symbol may be delisted
- CELG: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- LLL: No timezone found, symbol may be delisted
- SCG: No data found for this date range, symbol may be delisted
- VAR: No timezone found, symbol may be delisted
- ALXN: No timezone found, symbol may be delisted
- NFX: No data found for this date range, symbol may be delisted
- RTN: No timezone found, sy

C:\Users\Philip\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


(36, 452)
RRC    -0.377367
LUMN   -0.288038
CMG    -0.287391
M      -0.281005
SIG    -0.268821
TRIP   -0.257715
KMI    -0.247963
SRCL   -0.218816
PRGO   -0.210599
MAT    -0.182971
dtype: float64
['RRC', 'LUMN', 'CMG', 'M', 'SIG', 'TRIP', 'KMI', 'SRCL', 'PRGO', 'MAT']
-0.11921808577143563
[-0.14769898692233838, 0.3996787968108296, -0.04495962534446742, 0.15940032978101673, -0.11921808577143563]
['2016', 'MMM', 'BBBY', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'FSLR', 'TDC', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'MAT', 'URBN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'R', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'MON', 'GGP', 'AJG', 'AIZ', 'T', 'NFX', 'ADSK', 'AZO', 'AVB', 'AVY', 'SCG', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'VIAB', 'BRK-B', 'BBY', 'ADS', 'AET', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'RRC', 'NOV', 'BF-B', 'LE

...

(36, 365)
AIG    -1.278580
C      -0.907445
KEY    -0.603302
RF     -0.600363
ZION   -0.597150
ODP    -0.592657
HBAN   -0.573747
SLM    -0.486354
LEN    -0.447449
FITB   -0.441182
dtype: float64
['AIG', 'C', 'KEY', 'RF', 'ZION', 'ODP', 'HBAN', 'SLM', 'LEN', 'FITB']
0.40696873907937126
[-0.14769898692233838, 0.3996787968108296, -0.04495962534446742, 0.15940032978101673, -0.11921808577143563, 0.15515753777816654, 0.39659759277471024, -0.4257314298598701, -0.039758450681377316, 0.700061942044664, 0.3793355484974283, -0.23446749430007718, 0.40696873907937126]
['2008', 'MMM', 'BBBY', 'ABT', 'FII', 'MI', 'PLL', 'ADM', 'ADBE', 'ADP', 'FDO', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'SNDK', 'TE', 'FSLR', 'TDC', 'JCP', 'GAS', 'ALL', 'GOOG', 'MO', 'AMZN', 'MAT', 'URBN', 'AEE', 'AGN', 'AEP', 'AXP', 'AIG', 'AMT', 'CNX', 'AMP', 'ABC', 'CVG', 'AMGN', 'APH', 'ADI', 'R', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'MON', 'MOLX', 'CCE', 'AIZ', 'T', 'NYT', 'ADSK', 'AZO', 'AVB', 'AVY', 'SCG', 'BKR', 'BALL', 'BAC', 'BB

C:\Users\Philip\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


(36, 360)
AIG    -1.243570
AMD    -0.883631
MBI    -0.880274
GNW    -0.816254
ODP    -0.784961
EP     -0.687141
TGNA   -0.639192
THC    -0.632083
LEN    -0.628103
C      -0.613125
dtype: float64
['AIG', 'AMD', 'MBI', 'GNW', 'ODP', 'EP', 'TGNA', 'THC', 'LEN', 'C']
1.0292663891139375
[-0.14769898692233838, 0.3996787968108296, -0.04495962534446742, 0.15940032978101673, -0.11921808577143563, 0.15515753777816654, 0.39659759277471024, -0.4257314298598701, -0.039758450681377316, 0.700061942044664, 0.3793355484974283, -0.23446749430007718, 0.40696873907937126, 1.0292663891139375]
['2007', 'MMM', 'BBBY', 'ABT', 'FII', 'MI', 'PLL', 'ADM', 'ADBE', 'ADP', 'FDO', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'SNDK', 'TE', 'FSLR', 'TDC', 'JCP', 'GAS', 'ALL', 'GOOG', 'MO', 'AMZN', 'MAT', 'URBN', 'AEE', 'AGN', 'AEP', 'AXP', 'AIG', 'AMT', 'CNX', 'AMP', 'ABC', 'CVG', 'AMGN', 'APH', 'ADI', 'R', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'MON', 'MOLX', 'CCE', 'AIZ', 'T', 'NYT', 'ADSK', 'AZO', 'AVB', 'AVY', 'SCG', 'BKR', '

(36, 351)
MBI   -0.388040
BSX   -0.372451
LEN   -0.370370
PHM   -0.364300
AMD   -0.359016
JCI   -0.337294
BC    -0.334215
SLM   -0.309575
KBH   -0.274557
THC   -0.256921
dtype: float64
['MBI', 'BSX', 'LEN', 'PHM', 'AMD', 'JCI', 'BC', 'SLM', 'KBH', 'THC']
-0.7192004513693478
[-0.14769898692233838, 0.3996787968108296, -0.04495962534446742, 0.15940032978101673, -0.11921808577143563, 0.15515753777816654, 0.39659759277471024, -0.4257314298598701, -0.039758450681377316, 0.700061942044664, 0.3793355484974283, -0.23446749430007718, 0.40696873907937126, 1.0292663891139375, -0.7192004513693478]


C:\Users\Philip\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


After backtesting, the mean, annualised continuously compounding "holding period return" of each of the 11 strategy is as follows (all corrected to 3 significant figures):
1. Strategy 1: 0.122
2. Strategy 2: 0.132
3. Strategy 3: 0.135
4. Strategy 4: 0.0855
5. Strategy 5: 0.0565
6. Strategy 6: 0.0816
7. Strategy 7: 0.138
8. Srategy 8: 0.0638
9. Strategy 9: 0.0969
1. Strategy 10: 0.0732
2. Strategy 11: 0.0917

As a comparison, the annualised continuously compounding return of S&P 500 index is 0.0840. 

Please bear in mind that, a strategy with a mean return higher than that of S&P 500 index does not necessary mean that it is better than investing in S&P 500, because most of these strategies have a higher Beta and high volatility (standard deviation of return) than S&P 500. This means that the Sharpe ratio of these strategies may not be higher (actually, most of them has a lower Sharpe ratio) than that of S&P 500. 

In [702]:
with pd.ExcelWriter('Backtests_by_Python.xlsx') as writer:  
    st.to_excel(writer, sheet_name='s')
    ri.to_excel(writer, sheet_name='r')
    mean.to_excel(writer, sheet_name='miu')

In [602]:
fout.close()